# Connectome Manipulation : Connection Removal

## Introduction

Welcome to this demonstration on manipulating connectomes within SONATA circuit models. This Jupyter notebook serves as a hands-on guide to understanding and implementing techniques for the modification of neural circuitry, with a particular focus on the removal of connections. Our exploration is designed to equip researchers and practitioners with the necessary tools and knowledge to advance their work in computational neuroscience and related fields.

We will be removing 50% of connections between excitatory neurons in the circuit.

## Workflow

### Step 1: Load the connectome 

In [1]:
""" Global imports """
import json
import os
from pprint import pprint
from bluepysnap import Circuit

""" Local imports """

from connectome_manipulator.connectome_manipulation.manipulation import Manipulation
from pathlib import Path

In [15]:
# import circuit
circuit_path = '../../circuits/circuit_sonata_quick_scx_multi_circuit/' # adjust the circuit config path
circuit_config_path = f'{circuit_path}/circuit_config.json'
circuit_name = 'example'
node_set = 'Mosaic_A'
output_path = './output/'
working_dir = '.'

In [16]:
c = Circuit(circuit_config_path)

In [17]:
c.nodes._populations

{'NodeA': <bluepysnap.nodes.node_population.NodePopulation at 0x2aab7ecf30a0>,
 'NodeB': <bluepysnap.nodes.node_population.NodePopulation at 0x2aab7ed030a0>}

In [18]:
c.edges._populations

{'NodeA__NodeA__chemical': <bluepysnap.edges.edge_population.EdgePopulation at 0x2aab7eb7c1f0>,
 'NodeA__NodeB__chemical': <bluepysnap.edges.edge_population.EdgePopulation at 0x2aab7eb7cca0>,
 'NodeB__NodeA__chemical': <bluepysnap.edges.edge_population.EdgePopulation at 0x2aab7eb7c910>,
 'NodeB__NodeB__chemical': <bluepysnap.edges.edge_population.EdgePopulation at 0x2aab7e8b46d0>}

### Step 2: Create the Manipulation Config

Now that we know our node and edge populations in the circuit, we can start adressing them in the config file to specify what type of manipulation we want to perform

In [22]:
# Manipulation specifications
manip_config = {}

amount_pct = 20 # Amount of target neurons to rewire afferent connections to
edge_pop_name= 'NodeA__NodeA__chemical'

manip_config['edges_popul_name'] = edge_pop_name
manip_config['manip'] = {'name': f'{circuit_name}_remove_conns',
                         'fcts': [
                            {'source': 'conn_removal',
                            'sel_src': {'node_set': node_set},
                            'sel_dest': {'node_set': node_set},
                            'amount_pct': amount_pct,
                            'model_config':{}
                            }
                            ]}

## General settings
manip_config['circuit_config'] = circuit_config_path # SONATA (.json) format; path rel. to 'circuit_path'
manip_config['seed'] = 3220

In [23]:
pprint(manip_config)

{'circuit_config': '../../circuits/circuit_sonata_quick_scx_multi_circuit//circuit_config.json',
 'edges_popul_name': 'NodeA__NodeA__chemical',
 'manip': {'fcts': [{'amount_pct': 20,
                     'model_config': {},
                     'sel_dest': {'node_set': 'Mosaic_A'},
                     'sel_src': {'node_set': 'Mosaic_A'},
                     'source': 'conn_removal'}],
           'name': 'example_remove_conns'},
 'seed': 3220}


In [24]:
# Export model config for sbatch run

with open(os.path.join(working_dir, f'manip_config.json'), 'w') as f:
    json.dump(manip_config, f, indent=2)

### Step 3: Run the connectome manipulator

Now that we defined the manipulator config file, we can call the main method of the connectome manipulator from python. 

~~~
sbatch run_rewiring_parallel.sh "manip_config.json" "/path/to/output" 500
~~~

### Step 4: Examine the manipulated circuit

You can then use structural comparator feature of connectome manipulator to see effect of manipulation on the circuit

In [ ]:
structural_comparison_output_path = f'{output_path}/structcomp'
os.makedirs(structural_comparison_output_path,exist_ok=True)

In [ ]:
with open('structcomp_config__Orig_vs_Manip.json', 'r') as f:
    cfg_dict = json.load(f)
cfg_dict['working_dir'] = structural_comparison_output_path
cfg_dict['out_dir'] = structural_comparison_output_path
cfg_dict['circuits']['0']['circuit_config'] = circuit_config_path
cfg_dict['circuits']['1']['circuit_config'] = f'{output_path}/circuit_config.json'

# add edges_popul_name for each analysis

for plot_type in cfg_dict['plot_types']:
    plot_type['fct']['kwargs']['edges_popul_name'] = edge_pop_name

with open(f'{working_dir}/structcomp_config.json', 'w') as f:
    json.dump(cfg_dict, f, indent=2)

Examine the output directory for comparison and manipulated circuit. Then follow the other examples.